### Importing Packages

In [2]:
import pandas as pd 
import numpy as np 
import seaborn as sns 

import os
from loguru import logger

import warnings
warnings.filterwarnings('ignore')
import gc

### Common functions

In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
#         print(f'{col}:{col_type}')
        
        if col_type not in [object,'category']:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

### Importing Data

##### Importing train base data

In [4]:
for dirname, _, filenames in os.walk('/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/'):
    for filename in filenames:
        logger.info(f"Filename is:{filename}")

2024-02-29 06:57:15.163 | INFO     | __main__:<module>:3 - Filename is:train_credit_bureau_a_1_3.csv
2024-02-29 06:57:15.166 | INFO     | __main__:<module>:3 - Filename is:train_static_cb_0.csv
2024-02-29 06:57:15.167 | INFO     | __main__:<module>:3 - Filename is:train_applprev_1_0.csv
2024-02-29 06:57:15.168 | INFO     | __main__:<module>:3 - Filename is:train_person_2.csv
2024-02-29 06:57:15.169 | INFO     | __main__:<module>:3 - Filename is:train_base.csv
2024-02-29 06:57:15.172 | INFO     | __main__:<module>:3 - Filename is:train_tax_registry_a_1.csv
2024-02-29 06:57:15.174 | INFO     | __main__:<module>:3 - Filename is:train_static_0_0.csv
2024-02-29 06:57:15.175 | INFO     | __main__:<module>:3 - Filename is:train_credit_bureau_a_1_0.csv
2024-02-29 06:57:15.176 | INFO     | __main__:<module>:3 - Filename is:train_applprev_2.csv
2024-02-29 06:57:15.177 | INFO     | __main__:<module>:3 - Filename is:train_credit_bureau_a_2_6.csv
2024-02-29 06:57:15.178 | INFO     | __main__:<modul

## Base data

In [5]:
df_base = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_base.csv")

In [6]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1526659 entries, 0 to 1526658
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   case_id        1526659 non-null  int64 
 1   date_decision  1526659 non-null  object
 2   MONTH          1526659 non-null  int64 
 3   WEEK_NUM       1526659 non-null  int64 
 4   target         1526659 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 58.2+ MB


In [7]:
logger.info(f"Number of unique records: {df_base['case_id'].nunique()}")

2024-02-29 06:57:17.177 | INFO     | __main__:<module>:1 - Number of unique records: 1526659


In [8]:
df_base = reduce_mem_usage(df_base)

Memory usage of dataframe is 58.24 MB
Memory usage after optimization is: 17.49 MB
Decreased by 70.0%


### Datasets with depth 0 

#### Static data

In [9]:
df_static_0 = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_static_0_0.csv")
df_static_1 = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_static_0_1.csv")

In [10]:
df_static_0 = reduce_mem_usage(df_static_0)
df_static_1 = reduce_mem_usage(df_static_1)

Memory usage of dataframe is 1279.85 MB
Memory usage after optimization is: 361.68 MB
Decreased by 71.7%
Memory usage of dataframe is 666.73 MB
Memory usage after optimization is: 190.29 MB
Decreased by 71.5%


In [11]:
logger.info(f"Number of unique records in 0: {df_static_0['case_id'].nunique()}")
logger.info(f"Number of unique records in 1: {df_static_1['case_id'].nunique()}")

2024-02-29 06:58:22.530 | INFO     | __main__:<module>:1 - Number of unique records in 0: 1003757
2024-02-29 06:58:22.546 | INFO     | __main__:<module>:2 - Number of unique records in 1: 522902


In [12]:
## Combining the datasets
df_static = pd.concat([df_static_0,df_static_1],ignore_index=True)

In [13]:
df_static = reduce_mem_usage(df_static)

Memory usage of dataframe is 768.74 MB
Memory usage after optimization is: 552.16 MB
Decreased by 28.2%


In [14]:
logger.info(f"Total records in static dataset: {df_static['case_id'].nunique()}")

2024-02-29 06:58:32.537 | INFO     | __main__:<module>:1 - Total records in static dataset: 1526659


#### Static CB

In [15]:
df_static_cb = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_static_cb_0.csv")

In [16]:
logger.info(f"Number of unique records in 0: {df_static_cb['case_id'].nunique()}")

2024-02-29 06:58:44.693 | INFO     | __main__:<module>:1 - Number of unique records in 0: 1500476


In [17]:
df_static_cb = reduce_mem_usage(df_static_cb)
df_static_cb.info()

Memory usage of dataframe is 606.73 MB
Memory usage after optimization is: 159.58 MB
Decreased by 73.7%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500476 entries, 0 to 1500475
Data columns (total 53 columns):
 #   Column                   Non-Null Count    Dtype   
---  ------                   --------------    -----   
 0   case_id                  1500476 non-null  int32   
 1   assignmentdate_238D      136996 non-null   category
 2   assignmentdate_4527235D  114978 non-null   category
 3   assignmentdate_4955616D  71633 non-null    category
 4   birthdate_574D           607871 non-null   category
 5   contractssum_5085716L    157329 non-null   float32 
 6   dateofbirth_337D         1385691 non-null  category
 7   dateofbirth_342D         36500 non-null    category
 8   days120_123L             1385691 non-null  float16 
 9   days180_256L             1385691 non-null  float16 
 10  days30_165L              1385691 non-null  float16 
 11  days360_512L             1385691 non-

In [18]:
gc.collect()

0

##### Saving the new files 

In [25]:
!pwd

/kaggle/working


In [20]:
mkdir train

In [21]:
df_base.to_parquet('train/train_base.parquet')
df_static.to_parquet('train/train_static.parquet')
df_static_cb.to_parquet('train/train_static_cb.parquet')

### Datasets with Depth 1

In [26]:
df_applprev_1_0 = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_applprev_1_0.csv")

In [27]:
df_applprev_1_1 = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_applprev_1_1.csv")

In [31]:
df_applprev_1_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3887684 entries, 0 to 3887683
Data columns (total 41 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   case_id                      int64  
 1   actualdpd_943P               float64
 2   annuity_853A                 float64
 3   approvaldate_319D            object 
 4   byoccupationinc_3656910L     float64
 5   cancelreason_3545846M        object 
 6   childnum_21L                 float64
 7   creationdate_885D            object 
 8   credacc_actualbalance_314A   float64
 9   credacc_credlmt_575A         float64
 10  credacc_maxhisbal_375A       float64
 11  credacc_minhisbal_90A        float64
 12  credacc_status_367L          object 
 13  credacc_transactions_402L    float64
 14  credamount_590A              float64
 15  credtype_587L                object 
 16  currdebt_94A                 float64
 17  dateactivated_425D           object 
 18  district_544M                object 
 19  

In [32]:
df_applprev_1_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2638295 entries, 0 to 2638294
Data columns (total 41 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   case_id                      int64  
 1   actualdpd_943P               float64
 2   annuity_853A                 float64
 3   approvaldate_319D            object 
 4   byoccupationinc_3656910L     float64
 5   cancelreason_3545846M        object 
 6   childnum_21L                 float64
 7   creationdate_885D            object 
 8   credacc_actualbalance_314A   float64
 9   credacc_credlmt_575A         float64
 10  credacc_maxhisbal_375A       float64
 11  credacc_minhisbal_90A        float64
 12  credacc_status_367L          object 
 13  credacc_transactions_402L    float64
 14  credamount_590A              float64
 15  credtype_587L                object 
 16  currdebt_94A                 float64
 17  dateactivated_425D           object 
 18  district_544M                object 
 19  

In [33]:
df_applprev = pd.concat([df_applprev_1_0,df_applprev_1_1],ignore_index=True)

In [34]:
df_applprev = reduce_mem_usage(df_applprev)

Memory usage of dataframe is 2041.36 MB
Memory usage after optimization is: 592.79 MB
Decreased by 71.0%


In [55]:
df_applprev.to_parquet('/kaggle/working/train/train_applprev.parquet')